In [1]:
include("../src/main.jl");

In [2]:
# Random choice of target parameters
(a10, a11, a12, a13, a14, a20, a21, a22, a23, a24, a30, a31, a32, a33, a34, a40, a41, a42, a43, a44) =
    rand(1:100,20);

In [3]:
# Construct the modified system
HC.@var u1 u2 v1 v2 z11 z12 z21 z22 z31 z32 z41 z42 y1 y2

u_variables = [u1, u2]
v_variables = [v1, v2]

linear_part = [a10 + a11*u1 + a12*v1 + a13*z11 + a14*z12,
    a20 + a21*u1 + a22*v1 + a23*z21 + a24*z22,
    a30 + a31*u2 + a32*v2 + a33*z31 + a34*z32,
    a40 + a41*u2 + a42*v2 + a43*z41 + a44*z42]

z_variables = [z11, z12, z21, z22, z31, z32, z41, z42]
z_expressions = [u1*y1, u1*y2, v1*y1, v1*y2, u2*y1, u2*y2, v2*y1, v2*y2]

y_variables = [y1, y2]
y_expressions = [u1^2 + v1^2, u2^2 + v2^2]

F_modified = HC.System(vcat(linear_part, z_variables-z_expressions, y_variables-y_expressions), 
    variables=vcat(u_variables, v_variables, z_variables, y_variables)
)

System of length 14
 14 variables: u1, u2, v1, v2, z11, z12, z21, z22, z31, z32, z41, z42, y1, y2

 45 + 17*u1 + 26*v1 + 53*z11 + 52*z12
 52 + 18*u1 + 15*v1 + 40*z21 + 4*z22
 60 + 88*u2 + 2*v2 + 51*z31 + 54*z32
 21 + 81*u2 + 35*v2 + 46*z41 + 57*z42
 z11 - u1*y1
 z12 - u1*y2
 z21 - v1*y1
 z22 - v1*y2
 z31 - u2*y1
 z32 - u2*y2
 z41 - v2*y1
 z42 - v2*y2
 y1 - (u1^2 + v1^2)
 y2 - (u2^2 + v2^2)

In [4]:
# Polyhedral homotopies for the modified system
homotopies_modified, start_systems_modified = polyhedral_homotopies_and_start_systems(F_modified);

In [5]:
# Example of homotopy and start system
display(homotopies_modified[1])
display(start_systems_modified[1])

Homotopy in t of length 14
 14 variables: u1, u2, v1, v2, z11, z12, z21, z22, z31, z32, z41, z42, y1, y2

 45 + 53*z11 + 17*t^38*u1 + 26*t^820*v1 + 52*t^1932*z12
 18*u1 + 40*z21 + 15*t^722*v1 + 4*t^1968*z22 + 52*t^76
 60 + 2*v2 + 54*t^235*z32 + 88*t^267*u2 + 51*t^355*z31
 46*z41 + 57*z42 + 35*t^728*v2 + 81*t^2135*u2 + 21*t^1094
 z11 - u1*y1
 z12 - u1*y2
 z21 - v1*y1
 z22 - v1*y2
 z31 - u2*y1
 z32 - u2*y2
 z41 - v2*y1
 z42 - v2*y2
 y1 - t^2248*v1^2 - u1^2
 t^2404*y2 - u2^2 - v2^2

System of length 14
 14 variables: u1, u2, v1, v2, y1, y2, z11, z12, z21, z22, z31, z32, z41, z42

 45 + 53*z11
 18*u1 + 40*z21
 60 + 2*v2
 46*z41 + 57*z42
 z11 - u1*y1
 z12 - u1*y2
 z21 - v1*y1
 z22 - v1*y2
 z31 - u2*y1
 z32 - u2*y2
 z41 - v2*y1
 z42 - v2*y2
 y1 - u1^2
 -u2^2 - v2^2

In [6]:
# Back-substitute to homotopies for the original system
homotopies = HC.ModelKit.Homotopy[]
for H in homotopies_modified
    t = H.t
    H_new = HC.subs.(HC.subs.(H.expressions[1:length(linear_part)], z_variables=>z_expressions), y_variables=>y_expressions)
    push!(homotopies, HC.Homotopy(H_new, vcat(u_variables, v_variables), t))
end

In [7]:
# Example of homotopy
display(homotopies[1])

Homotopy in t of length 4
 4 variables: u1, u2, v1, v2

 45 + 17*t^38*u1 + 26*t^820*v1 + 53*(u1^2 + v1^2)*u1 + 52*t^1932*(u2^2 + v2^2)*u1
 18*u1 + 15*t^722*v1 + 40*(u1^2 + v1^2)*v1 + 4*t^1968*(u2^2 + v2^2)*v1 + 52*t^76
 60 + 2*v2 + 88*t^267*u2 + 54*t^235*(u2^2 + v2^2)*u2 + 51*t^355*(u1^2 + v1^2)*u2
 35*t^728*v2 + 81*t^2135*u2 + 46*(u1^2 + v1^2)*v2 + 57*(u2^2 + v2^2)*v2 + 21*t^1094